In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

cred = credentials.Certificate('kiosk-firestore-jnsy-bcfe4-firebase-adminsdk-269q4-4a795774bb.json')
firebase_admin.initialize_app(cred)

In [2]:
#모든 user를 루프돌아서
#선호도 가져오면 되겠다
#전체 아이템 알려면 menu도 루프 돌아야겠지
# tag='user'이면 모든 사용자 루프
def get_user_rating_list(tag='user'):
    docs = firestore.client().collection(tag).stream()
    doc_list = []
    
    for doc in docs:
        doc_list_row = []
        doc_list_row.append(doc.get('personUUID'))
        for key in doc.get('itemPreference').keys(): # {아이템이름str:점수int}
            doc_list_row.append([key,doc.get('itemPreference').get(key)])
            
        doc_list.append(doc_list_row)
    
    return doc_list


In [3]:
def data_preprocessing(user_items_list):
    temp_list = [] # user,item,rating 저장할 리스트
    
    for row in user_items_list: #[0]은 항상 person uuid
        if len(row) != 1: #길이가 하나인 행은 uuid만 있으므로 제외
            size = len(row)-1  # size는 user의 점수 매긴 item 개수
            for i in range(1,size+1): # 한 행 안의 아이템 루프 
                temp_list.append([row[0], row[i][0] , row[i][1] ])
                
    return temp_list
    

In [4]:
user_items_list = data_preprocessing(get_user_rating_list()) # order DB에서 데이터 가공해서 가져온다



print(user_items_list)

ResourceExhausted: 429 Quota exceeded.

In [ ]:
def userid_preprocessing(rating_item_df): #userid 전처리
    
    userid_encoder = LabelEncoder()
    users = rating_item_df['userid'].values.tolist()
    userid_encoder.fit(users)
    users_vec = userid_encoder.transform(users)
    
#print('인코딩 클래스:', userid_encoder.classes_)
#print('디코딩 원본 값:', encoder.inverse_transform([0,1])) #리스트나 스칼라로 전달하면 변환해서 반환함
    rating_item_df['userid'] = pd.Series(users_vec)
    
    return userid_encoder,rating_item_df # 사용자가 아이템에 점수 매긴 DF ## return

In [ ]:
rating_item_df = pd.DataFrame(user_items_list, columns=['userid','itemid','rating']) #ndarray로 변환
userid_encoder,rating_item_df = userid_preprocessing(rating_item_df)
print(type(userid_encoder.transform(['911283bd-2e4f-439b-89c4-fc8667782b51']).tolist()[0]))
#userid_encoder.classes_
#rating_item_df


In [ ]:
ratings_matrix = rating_item_df.pivot_table(values='rating', index='userid', columns='itemid').fillna(0)#,aggfunc='first')


#NaN 값을 모두 0으로 변환
#ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix # 행:사용자, 열:아이템 DF

In [ ]:
# 사용자/아이템 분기

In [ ]:
#아이템 간 유사도 행렬을 구하려면, 아이템이 행으로 나열되어 있어야 함
ratings_matrix_T = ratings_matrix.transpose()

ratings_matrix_T # 행:아이템, 열:사용자 DF

In [ ]:
## item cf
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)


item_sim_df  # 아이템-아이템 유사도 DF


In [ ]:
# df.sort_values(ascending=False)[:top_n]

#매개 item_sim_df 
def items_sim_dict(user_id,ratings_matrix,item_sim_df):
    item_sim_des_dict = {}
    for row,item in enumerate(item_sim_df):
        print(row, item) #item은 행:아이템 이름
        row_ndarray = item_sim_df.iloc[row].values
    #print(row_ndarray)
        des_sort_index = np.argsort(row_ndarray)[::-1][1:]
        print('!',des_sort_index)
        item_sim_des_list = item_sim_df.iloc[row].index[des_sort_index].tolist()
        print(item_sim_des_list)
    
        item_sim_des_dict[item] = item_sim_des_list
        
    return item_sim_des_dict
    #print(item_sim_des_dict)
    print()
    
print(item_sim_des_dict)
#인덱스0은 자기자신

# 각 아이템마다 자신과 가장 유사한 아이템 반환
# 사용자가 구매하지 않은?
item_sim_df
#np.argsort(item_sim_df.iloc[0].values)[::-1] #열 인덱스겠지

In [ ]:
## user cf

In [ ]:
def preference_df(ratings_matrix,item_sim_df):
    
    ratings_pred = ratings_matrix.values.dot(item_sim_df.values)/np.array([np.abs(item_sim_df.values).sum(axis=1)])
    df = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)
    
    return df
    
    

In [ ]:


ratings_pred_matrix = preference_df(ratings_matrix,item_sim_df)

ratings_pred_matrix  # 선호도 DF ?


In [ ]:
def item_cf(user_id,ratings_matrix,top_n=1): #매개 request 포함
    
    user_rating = ratings_matrix.loc[user_id,:] # 그래서 슬라이싱을 했구나
    already_ordered = user_rating[user_rating>0].index.tolist()
    order_list = ratings_matrix.columns.tolist() # 전체 메뉴판에서 가져올 필요가 없다 계산못하니까
    unordered_list = [item for item in order_list if item not in already_ordered]
    recomm_item = ratings_pred_matrix.loc[user_id, unordered_list].sort_values(ascending=False)[:top_n]
    
    return recomm_item #추천 아이템 1개
    

In [ ]:


item_cf(0,ratings_matrix)



In [ ]:


## 끝

In [ ]:
#def get_menu_list(tag='menu'):
#    docs = firestore.client().collection(tag).stream()
#    menu_list = []
#    
#    for doc in docs: # 메뉴 이름만 일단 모아서 반환한다
#        menu_list.append(doc.get('name'))
#    
#    return menu_list

In [ ]:
# 사용자 취향을 고려해 개인화된 추천을 수행해봅시다.

def predict_rating(ratings_arr, item_sim_arr):#평점, 유사도
    rating_pred = ratings_arr.dot(item_sim_arr)/np.array([np.abs(item_sim_arr).sum(axis=1)])
    #아 행으로 만드려고 [] 안에 sum 연산했구나. 결국 axis=0 같은 역할 아니야? 
    #응 아니야 []안하면 행으로 ndarray 만들어지니까
    
    # 평점*유사도/유사도합-각 아이템.
    return rating_pred


#먼저 사용자가 이미 평점을 준 영화를 제외하고 추천할 수 있도록
#평점을 주지 않은 영화를 리스트 객체로 반환하는 함수인 get_unseen_movies()를 생성합니다.

def get_unseen_movies(rating_df, userId): # 매개 사용자가 점수주지 않은 아이템 반환
    print('()', rating_df)
    
    all_menu_list = get_menu_list() #모든 메뉴 리스트 반환
    print(all_menu_list)

    #rating_item_list = rating_item_df.values.tolist()
    user_ordered_these_item = rating_df[ rating_df['userid']==userId ].values.tolist()
    print('user_ordered_these_item',user_ordered_these_item)
    
    if len(user_ordered_these_item) == 0:
        return None
    
    not_in_orderList = [x for x in all_menu_list if x not in user_ordered_these_item]
                        #rating_df['itemid'].values.tolist() 수정해야 한다
    
    return not_int_orderList
    

def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    #예측 평점 DataFrame에서 사용자id 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출해
    #가장 예측 평점이 높은 순으로 정렬한다
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    
    return recomm_movies